In [1]:
import datetime
import backtrader as bt
import backtrader.indicators as bi
import pandas as pd
import numpy as np

In [2]:
from lutils.stock import LTdxHq

In [3]:
class GridStrategy(bt.Strategy):
    params = (
        ("printlog", True),
        ("top", 12),
        ("buttom", 7),
    )
    
    def __init__(self):
        self.mid = (self.p.top + self.p.buttom)/2.0
        # 百分比区间计算
        #这里多1/2，是因为arange函数是左闭右开区间。
        perc_level = [x for x in np.arange(1 + 0.02 * 5, 1 - 0.02 * 5 - 0.02/2, -0.02)]
        # 价格区间
        # print(self.mid)
        self.price_levels = [self.mid * x for x in perc_level]
        # 记录上一次穿越的网格
        self.last_price_index = None
        # 总手续费
        self.comm = 0.0
 
    def next(self):
        # print(self.last_price_index)
        # 开仓
        if self.last_price_index == None:
            # print("b", len(self.price_levels))
            for i in range(len(self.price_levels)):
                price = self.data.close[0]
                # print("c", i, price, self.price_levels[i][0])
                if self.data.close[0] > self.price_levels[i]:
                    self.last_price_index = i
                    self.order_target_percent(target=i/(len(self.price_levels) - 1))
                    print("a")
                    return
        # 调仓
        else:
            signal = False
            while True:
                upper = None
                lower = None
                if self.last_price_index > 0:
                    upper = self.price_levels[self.last_price_index - 1]
                if self.last_price_index < len(self.price_levels) - 1:
                    lower = self.price_levels[self.last_price_index + 1]
                # 还不是最轻仓，继续涨，再卖一档
                if upper != None and self.data.close > upper:
                    self.last_price_index = self.last_price_index - 1
                    signal = True
                    continue
                # 还不是最重仓，继续跌，再买一档
                if lower != None and self.data.close < lower:
                    self.last_price_index = self.last_price_index + 1
                    signal = True
                    continue
                break
            if signal:
                self.long_short = None
                self.order_target_percent(target=self.last_price_index/(len(self.price_levels) - 1))
 
    # 输出交易记录
    def log(self, txt, dt = None, doprint = False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
 
    def notify_order(self, order):
        # 有交易提交/被接受，啥也不做
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 交易完成，报告结果
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('执行买入, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.comm += order.executed.comm
            else:
                self.log('执行卖出, 价格: %.2f, 成本: %.2f, 手续费 %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.comm += order.executed.comm
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("交易失败")
        self.order = None
 
    # 输出手续费
    def stop(self):
        self.log("手续费:%.2f 成本比例:%.5f" % (self.comm, self.comm/self.broker.getvalue()))



In [4]:
lt = LTdxHq()

In [5]:
code = '000404' # 510300 000032 300142 603636 600519 688567 000100 000404

In [6]:
df = lt.get_k_data_daily(code, qfq=True)

In [7]:
df = df['2021-01-01':]

In [8]:
top = df.close.min() * .8
buttom = df.close.max() * 1.1

In [9]:
top

2.824

In [10]:
buttom

5.324

In [11]:
df.index = pd.to_datetime(df.index)

In [12]:
cerebro = bt.Cerebro(oldtrades=True)

feed = bt.feeds.PandasData(dataname=df, openinterest=None, compression=1, timeframe=bt.TimeFrame.Minutes)
cerebro.adddata(feed)

# top = 4.2
# buttom = 3.5
cerebro.addstrategy(GridStrategy, top=top, buttom=buttom)

# 小场面1万起始资金
cerebro.broker.setcash(10000.0)

# 手续费万5
cerebro.broker.setcommission(0.0005)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

result = cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
a
2021-01-11, 执行买入, 价格: 4.34, 成本: 998.20, 手续费 0.50
2021-01-12, 执行买入, 价格: 4.13, 成本: 3035.55, 手续费 1.52
2021-01-15, 执行买入, 价格: 4.04, 成本: 1030.20, 手续费 0.52
2021-01-19, 执行卖出, 价格: 4.16, 成本: 1075.05, 手续费 0.54
2021-01-25, 执行买入, 价格: 4.04, 成本: 1062.52, 手续费 0.53
2021-01-28, 执行卖出, 价格: 4.14, 成本: 1093.65, 手续费 0.55
2021-01-29, 执行卖出, 价格: 4.27, 成本: 1019.36, 手续费 0.53
2021-02-01, 执行买入, 价格: 4.12, 成本: 1083.56, 手续费 0.54
2021-02-03, 执行买入, 价格: 4.02, 成本: 1053.24, 手续费 0.53
2021-02-04, 执行买入, 价格: 3.92, 成本: 1034.88, 手续费 0.52
2021-02-05, 执行买入, 价格: 3.81, 成本: 1992.63, 手续费 1.00
2021-02-18, 执行卖出, 价格: 3.96, 成本: 1056.88, 手续费 0.52
2021-02-19, 执行卖出, 价格: 4.00, 成本: 1032.86, 手续费 0.52
2021-02-22, 执行卖出, 价格: 4.14, 成本: 1060.88, 手续费 0.55
2021-02-23, 执行卖出, 价格: 4.19, 成本: 1004.83, 手续费 0.53
2021-03-01, 执行买入, 价格: 4.10, 成本: 1086.50, 手续费 0.54
2021-03-02, 执行卖出, 价格: 4.19, 成本: 1058.26, 手续费 0.55
2021-03-04, 执行卖出, 价格: 4.21, 成本: 1026.07, 手续费 0.54
2021-03-09, 执行买入, 价格: 4.14, 成本: 1068.12, 手续费 0.53
2021-03-11, 执行

In [13]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2020, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 1707x960 with 4 Axes>]]

In [8]:
# start = "2018-01-01"
# end = "2021-07-05"
# name = ["300etf"]
# code = ["510300"]
# backtest = backtest.BackTest(GridStrategy, start, end, code, name, 100000)
# result = backtest.run()
# # backtest.output()
# print(result)